![](images/ciphers_logo.gif){fig-align="center" width="428"}

# Importing Data to SQL


In [ ]:
#| echo: false
#| code-fold: true
#| code-summary: "Show the code"

import pandas as pd
import mysql.connector
from mysql.connector import Error


def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Database connection established successfully.")
    except Error as e:
        print(f"Error: {e}")

    return connection


def insert_data(connection, query, data):
    cursor = connection.cursor()
    try:
        cursor.execute(query, data)
        connection.commit()
        print("Data inserted successfully.")
    except Error as e:
        print(f"Error: {e}")


file_path_students = '/Users/ASUS/students.xlsx'
file_path_company = '/Users/ASUS/company.xlsx'
file_path_internship = '/Users/ASUS/internship.xlsx'
file_path_rating = '/Users/ASUS/rating.xlsx'
data1 = pd.read_excel(file_path_students, sheet_name=None)
data2 = pd.read_excel(file_path_company, sheet_name=None)
data3 = pd.read_excel(file_path_internship, sheet_name=None)
data4 = pd.read_excel(file_path_rating, sheet_name=None)
connection = create_connection("127.0.0.1", "root", "415", "emu415")

# Adding Students to the Database
for index, row in data1['students'].iterrows():
    insert_query = "INSERT INTO students (student_id, first_name, last_name, email, join_year, gpa) VALUES (%s, %s, " \
                   "%s, %s, %s, %s)"
    student_data = (row['student_id'], row['first_name'], row['last_name'], row['email'], row['join_year'], row['gpa'])
    insert_data(connection, insert_query, student_data)

# Adding Companies to the Database
for index, row in data2['company'].iterrows():
    insert_query = "INSERT INTO companies (company_id, company_name, industry, location, contact_info, website) " \
                   "VALUES (%s, %s, %s, %s, %s, %s)"
    company_data = (
        row['company_id'], row['company_name'], row['industry'], row['location'], row['contact_info'], row['website'])
    insert_data(connection, insert_query, company_data)

# Adding Internships to the Database
for index, row in data3['internship'].iterrows():
    print(row['internship_id'])
    insert_query = "INSERT INTO internships (internship_id, student_id, company_id, intern_type, department, year, " \
                   "start_date, end_date, duration, income, working_hours, transportation_status) VALUES (%s, %s, %s, " \
                   "%s, " \
                   "%s, %s, %s, %s, %s, %s, %s, %s)"
    internship_data = (
        row['internship_id'], row['student_id'], row['company_id'], row['intern_type'], row['department'], row['year'],
        str(row['start_date']), str(row['end_date']), row['duration'], row['income'], row['working_hours'],
        row['transportation_status'])
    insert_data(connection, insert_query, internship_data)

# Adding Ratings to the Database
for index, row in data4['rating'].iterrows():
    # Check if the comment is empty, if so, replace it with None
    comment = row['comment'] if pd.notna(row['comment']) else None
    insert_query = "INSERT INTO ratings (rating_id, internship_id, transportation_rating ,work_env_rating, " \
                   "learning_opp_rating, food_rating, hr_rating, mentor_rating, comment) VALUES (%s, %s, %s, %s, %s, " \
                   "%s, %s, %s, %s)"
    rating_data = (row['rating_id'], row['internship_id'], row['transportation_rating'], row['work_env_rating'],
                   row['learning_opp_rating'], row['food_rating'], row['hr_rating'], row['mentor_rating'],
                   comment)  # Use the processed comment variable
    insert_data(connection, insert_query, rating_data)

# Closing the Database Connection
if connection:
    connection.close()